In [1]:
### CALCULATE ACCESSIBILITY FOR THE MURD ALLOSTERIC POCKETS IN 4 FRAG STRUCTURES ####

In [14]:
from subprocess import Popen, PIPE
from scipy.spatial import distance
from collections import Counter
import matplotlib.pyplot as plt
from Bio.SeqUtils import seq1
from Bio.PDB import *
from rdkit import Chem
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import tqdm
import pickle
import os
import time
import shutil
import gzip
import tarfile
import sys
import random
import pybel
from rdkit.Chem import rdFreeSASA

In [15]:
def SASA(prot, lig): 

    # Protonation gives too many issues. Avoid it
    
    #compute ligand SASA
    #lig_h = Chem.rdmolops.AddHs(lig, addCoords=True, explicitOnly=True)
    lig_h = lig
    # Get Van der Waals radii (angstrom)
    ptable = Chem.GetPeriodicTable()
    radii = [ptable.GetRvdw(atom.GetAtomicNum()) for atom in lig_h.GetAtoms()]
    # Compute solvent accessible surface area
    lig_sasa = rdFreeSASA.CalcSASA(lig_h, radii)

    # Join protein & ligand
    comp = Chem.CombineMols(prot, lig)
    comp_h = comp
    #comp_h = Chem.AddHs(comp, addCoords=True)
    # Get Van der Waals radii (angstrom)
    ptable = Chem.GetPeriodicTable()
    radii = [ptable.GetRvdw(atom.GetAtomicNum()) for atom in comp_h.GetAtoms()]
    # Compute solvent accessible surface area
    comp_sasa = rdFreeSASA.CalcSASA(comp_h, radii)
    comp_lig = Chem.GetMolFrags(comp_h, asMols=True,  sanitizeFrags=True)
    comp_lig = [i for i in comp_lig if lig_h.GetNumHeavyAtoms() == i.GetNumHeavyAtoms()][0]
    
    lig_sasa_free = 0
    for a in lig_h.GetAtoms():
        lig_sasa_free += float(a.GetProp("SASA"))

    lig_sasa_bound = 0
    for a in comp_lig.GetAtoms():
        lig_sasa_bound += float(a.GetProp("SASA"))
        
    return round(lig_sasa_free, 3), round(lig_sasa_bound, 3)

In [16]:
path = "../data/structures/"

sts = ['SagaMurD_Frag349', "SagaMurD_Frag373", "SagaMurD_Frag374", "SagaMurD_Frag378"]

In [25]:
for st in sts:
    
    # Get SASA
    prot = Chem.MolFromPDBFile(os.path.join(path, st + ".pdb"))
    lig = Chem.MolFromPDBFile(os.path.join(path, st.split("Frag")[1] + "_ligand_only.pdb"))
    
    lig_sasa_free, lig_sasa_bound = SASA(prot, lig)
    
    print(st + "\t" + str(round(lig_sasa_bound/lig_sasa_free, 3)))

SagaMurD_Frag349	0.46
SagaMurD_Frag373	0.457
SagaMurD_Frag374	0.342
SagaMurD_Frag378	0.473
